In [ ]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

In [ ]:
tabelas_cvm = read_sql("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA='cvm'")
for _, x in tabelas_cvm.iterrows():
    table_name = x.iloc[0]
    # if 'fi_cad' in table_name:
    if 'cda_fi' in table_name:
        df = db.read_sql("SELECT * FROM cvm.{} LIMIT 10".format(table_name))
        print(table_name, 'Colunas: ', df.columns.tolist())
        # df = db.read_sql("SELECT * FROM cvm.{} WHERE cnpj_fundo = '42.195.812/0001-80'".format(table_name))
        # print(table_name, 'Linhas: ', len(df))
        # df = db.read_sql("SELECT * FROM cvm.{} WHERE cnpj_fundo = '42.195.812/0001-80'".format(table_name))
        # print(table_name, 'Linhas: ', len(df))

In [ ]:
sql("DROP VIEW IF EXISTS cvm.cotas CASCADE;")
sql("""CREATE OR REPLACE VIEW cvm.cotas AS
SELECT COALESCE(tp_fundo, tp_fundo_classe) tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) cnpj_fundo, dt_comptc, vl_total, vl_quota, vl_patrim_liq, captc_dia, resg_dia, nr_cotst, id_subclasse
FROM cvm.fi_doc_inf_diario_inf_diario_fi;""")

In [ ]:
sql("DROP VIEW IF EXISTS cvm.cadastro CASCADE;")
sql("""
CREATE OR REPLACE VIEW cvm.cadastro AS
SELECT 
    -- Chaves Identificadoras
    h_social.cnpj_fundo,
    h_social.dt_reg,

    -- Lógica de Datas (Início)
    GREATEST(
        h_social.dt_ini_denom_social::date,
        COALESCE(h_admin.dt_ini_admin::date, '1900-01-01'),
        COALESCE(h_gestor.dt_ini_gestor::date, '1900-01-01'),
        COALESCE(h_sit.dt_ini_sit::date, '1900-01-01')
    ) AS dt_ini,

    -- Administração, Auditoria e Custódia
    h_admin.cnpj_admin,
    h_admin.admin,
    h_auditor.cnpj_auditor,
    h_auditor.auditor,
    h_custos.cnpj_custodiante,
    h_custos.custodiante,

    -- Características do Fundo
    h_classe.classe,
    h_condom.condom,
    h_control.cnpj_controlador,
    h_control.controlador,
    h_comerc.denom_comerc,
    h_social.denom_social,
    h_diretor.diretor,

    -- Flags e Tipos
    h_excl.fundo_exclusivo,
    h_fic.fundo_cotas,

    -- Gestão
    h_gestor.pf_pj_gestor,
    h_gestor.cpf_cnpj_gestor,
    h_gestor.gestor,

    -- Público, Rentabilidade e Situação
    h_publ.publico_alvo,
    h_rent.rentab_fundo,
    h_sit.sit,

    -- Taxas
    h_taxa_adm.taxa_adm,
    h_taxa_adm.inf_taxa_adm,
    h_taxa_perf.vl_taxa_perfm,
    h_taxa_perf.ds_taxa_perfm,
    h_trib.trib_lprazo,

    -- Lógica de Data de Fim Consolidada
    -- NULLIF(..., '') garante que strings vazias não quebrem o IS NULL
    LEAST(
        NULLIF(h_social.dt_fim_denom_social, '')::date,
        NULLIF(h_admin.dt_fim_admin, '')::date,
        NULLIF(h_auditor.dt_fim_auditor, '')::date,
        NULLIF(h_classe.dt_fim_classe, '')::date,
        NULLIF(h_condom.dt_fim_condom, '')::date,
        NULLIF(h_control.dt_fim_controlador, '')::date,
        NULLIF(h_custos.dt_fim_custodiante, '')::date,
        NULLIF(h_comerc.dt_fim_denom_comerc, '')::date,
        NULLIF(h_diretor.dt_fim_diretor, '')::date,
        NULLIF(h_excl.dt_fim_st_exclusivo, '')::date,
        NULLIF(h_fic.dt_fim_st_cotas, '')::date,
        NULLIF(h_gestor.dt_fim_gestor, '')::date,
        NULLIF(h_publ.dt_fim_publico_alvo, '')::date,
        NULLIF(h_rent.dt_fim_rentab, '')::date,
        NULLIF(h_sit.dt_fim_sit, '')::date,
        NULLIF(h_trib.dt_fim_st_trib_lprazo, '')::date
    ) AS dt_fim

FROM cvm.fi_cad_fi_hist_denom_social h_social
LEFT JOIN cvm.fi_cad_fi_hist_admin h_admin ON h_social.cnpj_fundo = h_admin.cnpj_fundo AND h_social.dt_reg = h_admin.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_auditor h_auditor ON h_social.cnpj_fundo = h_auditor.cnpj_fundo AND h_social.dt_reg = h_auditor.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_classe h_classe ON h_social.cnpj_fundo = h_classe.cnpj_fundo AND h_social.dt_reg = h_classe.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_condom h_condom ON h_social.cnpj_fundo = h_condom.cnpj_fundo AND h_social.dt_reg = h_condom.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_controlador h_control ON h_social.cnpj_fundo = h_control.cnpj_fundo AND h_social.dt_reg = h_control.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_custodiante h_custos ON h_social.cnpj_fundo = h_custos.cnpj_fundo AND h_social.dt_reg = h_custos.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_denom_comerc h_comerc ON h_social.cnpj_fundo = h_comerc.cnpj_fundo AND h_social.dt_reg = h_comerc.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_diretor_resp h_diretor ON h_social.cnpj_fundo = h_diretor.cnpj_fundo AND h_social.dt_reg = h_diretor.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_exclusivo h_excl ON h_social.cnpj_fundo = h_excl.cnpj_fundo AND h_social.dt_reg = h_excl.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_fic h_fic ON h_social.cnpj_fundo = h_fic.cnpj_fundo AND h_social.dt_reg = h_fic.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_gestor h_gestor ON h_social.cnpj_fundo = h_gestor.cnpj_fundo AND h_social.dt_reg = h_gestor.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_publico_alvo h_publ ON h_social.cnpj_fundo = h_publ.cnpj_fundo AND h_social.dt_reg = h_publ.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_rentab h_rent ON h_social.cnpj_fundo = h_rent.cnpj_fundo AND h_social.dt_reg = h_rent.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_sit h_sit ON h_social.cnpj_fundo = h_sit.cnpj_fundo AND h_social.dt_reg = h_sit.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_taxa_adm h_taxa_adm ON h_social.cnpj_fundo = h_taxa_adm.cnpj_fundo AND h_social.dt_reg = h_taxa_adm.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_taxa_perfm h_taxa_perf ON h_social.cnpj_fundo = h_taxa_perf.cnpj_fundo AND h_social.dt_reg = h_taxa_perf.dt_reg
LEFT JOIN cvm.fi_cad_fi_hist_trib_lprazo h_trib ON h_social.cnpj_fundo = h_trib.cnpj_fundo AND h_social.dt_reg = h_trib.dt_reg;
""")

In [ ]:
# Mapeamento exato das colunas baseado no seu log
cda_configs = {
    "cda_fie": {
        "table": "fi_doc_cda_fie",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, dt_comptc, vl_patrim_liq, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, cd_ativo, ds_ativo, dt_venc, pf_pj_emissor, cpf_cnpj_emissor, emissor, risco_emissor, cd_selic, dt_ini_vigencia, cd_pais, pais, cd_bv_merc, bv_merc"
    },
    "cda_fie_confid": {
        "table": "fi_doc_cda_fie_confid",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic"
    },
    "cda_fiim": {
        "table": "fi_doc_cda_fiim",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, vl_patrim_liq, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, cd_ativo, ds_ativo, dt_venc, pf_pj_emissor, cpf_cnpj_emissor, emissor, risco_emissor, cd_selic, dt_ini_vigencia, cd_pais, pais, cd_bv_merc, bv_merc"
    },
    "cda_fiim_confid": {
        "table": "fi_doc_cda_fiim_confid",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic"
    },
    "cda_fi_blc_1": {
        "table": "fi_doc_cda_fi_blc_1",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, tp_titpub, cd_isin, cd_selic, dt_emissao, dt_venc"
    },
    "cda_fi_blc_3": {
        "table": "fi_doc_cda_fi_blc_3",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, cd_swap, ds_swap"
    },
    "cda_fi_blc_4": {
        "table": "fi_doc_cda_fi_blc_4",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, cd_ativo, ds_ativo, cd_isin, dt_ini_vigencia, dt_fim_vigencia"
    },
    "cda_fi_blc_5": {
        "table": "fi_doc_cda_fi_blc_5",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, cnpj_emissor, emissor, dt_venc, titulo_posfx, cd_indexador_posfx, ds_indexador_posfx, pr_indexador_posfx, pr_cupom_posfx, pr_taxa_prefx, risco_emissor, ag_risco, dt_risco, grau_risco"
    },
    "cda_fi_blc_6": {
        "table": "fi_doc_cda_fi_blc_6",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, pf_pj_emissor, cpf_cnpj_emissor, emissor, dt_venc, titulo_posfx, cd_indexador_posfx, ds_indexador_posfx, pr_indexador_posfx, pr_cupom_posfx, pr_taxa_prefx, titulo_cetip, titulo_garantia, cnpj_instituicao_financ_coobr"
    },
    "cda_fi_blc_7": {
        "table": "fi_doc_cda_fi_blc_7",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, invest_coletivo, invest_coletivo_gestor, emissor, dt_venc, cd_pais, pais, cd_bv_merc, bv_merc, cd_ativo_bv_merc, risco_emissor, ag_risco, dt_risco, grau_risco, ds_ativo_exterior, qt_ativo_exterior, vl_ativo_exterior"
    },
    "cda_fi_blc_8": {
        "table": "fi_doc_cda_fi_blc_8",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, tp_ativo, emissor_ligado, tp_negoc, qt_venda_negoc, vl_venda_negoc, qt_aquis_negoc, vl_aquis_negoc, qt_pos_final, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic, ds_ativo, pf_pj_emissor, cpf_cnpj_emissor, emissor"
    },
    "cda_fi_confid": {
        "table": "fi_doc_cda_fi_confid",
        "cols": "tp_fundo_classe AS tp_fundo, cnpj_fundo_classe AS cnpj_fundo, denom_social, dt_comptc, tp_aplic, vl_venda_negoc, vl_aquis_negoc, vl_merc_pos_final, vl_custo_pos_final, dt_confid_aplic"
    },
    "cda_fi_pl": {
        "table": "fi_doc_cda_fi_pl",
        "cols": "COALESCE(tp_fundo, tp_fundo_classe) AS tp_fundo, COALESCE(cnpj_fundo, cnpj_fundo_classe) AS cnpj_fundo, denom_social, dt_comptc, vl_patrim_liq"
    }
}

for view_name, info in cda_configs.items():
    sql(f"DROP VIEW IF EXISTS cvm.{view_name} CASCADE;")
    
    query = f"""
    CREATE VIEW cvm.{view_name} AS
    SELECT {info['cols']}
    FROM cvm.{info['table']};
    """
    sql(query)